# exp13スコア差の調査

**目的**: ValidationとTestでスコアが大きく異なる原因を特定する

**観測されたスコア**:
- Validation: WeightedEnsemble_A MAE=42.61, RMSE=61.14, R2=0.626, WAPE=31.66%
- Test: WeightedEnsemble_A MAE=27.05, RMSE=34.25, R2=0.663, WAPE=28.20%

**調査項目**:
1. データ分割の詳細（期間・件数）
2. 目的変数の分布比較
3. 時系列トレンドの確認
4. 外部要因の影響

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定（オプション）
plt.rcParams['font.sans-serif'] = ['MS Gothic', 'Yu Gothic', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## Step 1: データの読み込みと前処理

In [ ]:
# データ読み込み
calender = pd.read_csv('../input/calender_data.csv')
cm_data = pd.read_csv('../input/cm_data.csv')
gt_service = pd.read_csv('../input/gt_service_name.csv')
acc_get = pd.read_csv('../input/regi_acc_get_data_transform.csv')
call_data = pd.read_csv('../input/regi_call_data_transform.csv')

# 日付型に変換
calender['cdr_date'] = pd.to_datetime(calender['cdr_date'])
cm_data['cdr_date'] = pd.to_datetime(cm_data['cdr_date'])
acc_get['cdr_date'] = pd.to_datetime(acc_get['cdr_date'])
call_data['cdr_date'] = pd.to_datetime(call_data['cdr_date'])
gt_service['week'] = pd.to_datetime(gt_service['week'])

print(f"データ期間: {call_data['cdr_date'].min()} ~ {call_data['cdr_date'].max()}")
print(f"全日数: {(call_data['cdr_date'].max() - call_data['cdr_date'].min()).days}日")

In [ ]:
# データマージ
df = call_data.copy()
df = df.merge(calender, on='cdr_date', how='left')
df = df.merge(cm_data, on='cdr_date', how='left')
df = df.merge(acc_get, on='cdr_date', how='left')

# Google Trendsを日次に展開
gt_service_daily = []
for idx, row in gt_service.iterrows():
    week_start = row['week']
    for i in range(7):
        date = week_start + timedelta(days=i)
        gt_service_daily.append({
            'cdr_date': date,
            'search_cnt': row['search_cnt']
        })

gt_daily = pd.DataFrame(gt_service_daily)
df = df.merge(gt_daily, on='cdr_date', how='left')

# 翌日の入電数を目的変数にする
df['target_next_day'] = df['call_num'].shift(-1)
df = df.dropna(subset=['target_next_day']).reset_index(drop=True)

# 平日のみ
df_model = df[df['dow'].isin([1, 2, 3, 4, 5])].copy().reset_index(drop=True)

print(f"\n平日データ数: {len(df_model)}行")
print(f"期間: {df_model['cdr_date'].min()} ~ {df_model['cdr_date'].max()}")

## Step 2: データ分割（exp13と同じ方法）

In [ ]:
val_months = 2
test_months = 2

max_date = df_model['cdr_date'].max()
test_start = max_date - pd.Timedelta(days=30*test_months)
val_start = test_start - pd.Timedelta(days=30*val_months)

train_df = df_model[df_model['cdr_date'] < val_start].copy()
val_df = df_model[(df_model['cdr_date'] >= val_start) & (df_model['cdr_date'] < test_start)].copy()
test_df = df_model[df_model['cdr_date'] >= test_start].copy()

print("="*80)
print("データ分割の詳細")
print("="*80)

print(f"\nTrain:")
print(f"  期間: {train_df['cdr_date'].min()} ~ {train_df['cdr_date'].max()}")
print(f"  件数: {len(train_df)}行")
print(f"  日数: {(train_df['cdr_date'].max() - train_df['cdr_date'].min()).days}日")

print(f"\nValidation:")
print(f"  期間: {val_df['cdr_date'].min()} ~ {val_df['cdr_date'].max()}")
print(f"  件数: {len(val_df)}行")
print(f"  日数: {(val_df['cdr_date'].max() - val_df['cdr_date'].min()).days}日")
print(f"  ※ {val_df['cdr_date'].min().strftime('%Y年%m月')} ~ {val_df['cdr_date'].max().strftime('%Y年%m月')}")

print(f"\nTest:")
print(f"  期間: {test_df['cdr_date'].min()} ~ {test_df['cdr_date'].max()}")
print(f"  件数: {len(test_df)}行")
print(f"  日数: {(test_df['cdr_date'].max() - test_df['cdr_date'].min()).days}日")
print(f"  ※ {test_df['cdr_date'].min().strftime('%Y年%m月')} ~ {test_df['cdr_date'].max().strftime('%Y年%m月')}")

## Step 3: 目的変数の分布比較

In [ ]:
print("="*80)
print("目的変数（翌日入電数）の統計")
print("="*80)

print("\nTrain:")
print(train_df['target_next_day'].describe())

print("\nValidation:")
print(val_df['target_next_day'].describe())

print("\nTest:")
print(test_df['target_next_day'].describe())

# 平均値の比較
train_mean = train_df['target_next_day'].mean()
val_mean = val_df['target_next_day'].mean()
test_mean = test_df['target_next_day'].mean()

print("\n="*80)
print("平均値の比較")
print("="*80)
print(f"Train     : {train_mean:.2f}")
print(f"Validation: {val_mean:.2f} (Train比: {(val_mean - train_mean) / train_mean * 100:+.1f}%)")
print(f"Test      : {test_mean:.2f} (Train比: {(test_mean - train_mean) / train_mean * 100:+.1f}%)")
print(f"\n🔥 Validation vs Test の差: {val_mean - test_mean:.2f} ({(val_mean - test_mean) / test_mean * 100:.1f}%)")

## Step 4: 時系列の可視化

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# 1. 生データのプロット
ax = axes[0]
ax.plot(train_df['cdr_date'], train_df['target_next_day'], 
        label='Train', alpha=0.6, linewidth=1, color='blue')
ax.plot(val_df['cdr_date'], val_df['target_next_day'], 
        label='Validation', alpha=0.8, linewidth=1.5, color='orange')
ax.plot(test_df['cdr_date'], test_df['target_next_day'], 
        label='Test', alpha=0.8, linewidth=1.5, color='green')
ax.axvline(val_start, color='orange', linestyle='--', alpha=0.5, linewidth=2)
ax.axvline(test_start, color='green', linestyle='--', alpha=0.5, linewidth=2)
ax.set_xlabel('Date')
ax.set_ylabel('Target (Next Day Calls)')
ax.set_title('Time Series: Target Distribution (Raw Data)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# 2. 7日移動平均
ax = axes[1]
train_ma = train_df.set_index('cdr_date')['target_next_day'].rolling(window=7, center=False).mean()
val_ma = val_df.set_index('cdr_date')['target_next_day'].rolling(window=7, center=False).mean()
test_ma = test_df.set_index('cdr_date')['target_next_day'].rolling(window=7, center=False).mean()

ax.plot(train_ma.index, train_ma.values, label='Train (7-day MA)', linewidth=2, color='blue')
ax.plot(val_ma.index, val_ma.values, label='Validation (7-day MA)', linewidth=2, color='orange')
ax.plot(test_ma.index, test_ma.values, label='Test (7-day MA)', linewidth=2, color='green')
ax.axvline(val_start, color='orange', linestyle='--', alpha=0.5, linewidth=2)
ax.axvline(test_start, color='green', linestyle='--', alpha=0.5, linewidth=2)
ax.axhline(train_mean, color='blue', linestyle=':', alpha=0.5, label=f'Train Mean={train_mean:.1f}')
ax.axhline(val_mean, color='orange', linestyle=':', alpha=0.5, label=f'Val Mean={val_mean:.1f}')
ax.axhline(test_mean, color='green', linestyle=':', alpha=0.5, label=f'Test Mean={test_mean:.1f}')
ax.set_xlabel('Date')
ax.set_ylabel('Target (7-day Moving Average)')
ax.set_title('Time Series: Trend Comparison (7-day MA)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# 3. ヒストグラム比較
ax = axes[2]
ax.hist(train_df['target_next_day'], bins=30, alpha=0.5, label=f'Train (mean={train_mean:.1f})', color='blue')
ax.hist(val_df['target_next_day'], bins=30, alpha=0.5, label=f'Validation (mean={val_mean:.1f})', color='orange')
ax.hist(test_df['target_next_day'], bins=30, alpha=0.5, label=f'Test (mean={test_mean:.1f})', color='green')
ax.set_xlabel('Target (Next Day Calls)')
ax.set_ylabel('Frequency')
ax.set_title('Distribution Comparison', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../output/exp13/score_diff_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()

print("可視化を保存: ../output/exp13/score_diff_timeseries.png")

## Step 5: 月別・曜日別の分析

In [ ]:
print("="*80)
print("月別・曜日別の特徴")
print("="*80)

print("\n【Validation期間の特徴】")
print("\n月別平均:")
val_month = val_df.groupby('month')['target_next_day'].agg(['mean', 'std', 'count'])
print(val_month)

print("\n曜日別平均 (1=月, 2=火, 3=水, 4=木, 5=金):")
val_dow = val_df.groupby('dow')['target_next_day'].agg(['mean', 'std', 'count'])
print(val_dow)

print("\n【Test期間の特徴】")
print("\n月別平均:")
test_month = test_df.groupby('month')['target_next_day'].agg(['mean', 'std', 'count'])
print(test_month)

print("\n曜日別平均 (1=月, 2=火, 3=水, 4=木, 5=金):")
test_dow = test_df.groupby('dow')['target_next_day'].agg(['mean', 'std', 'count'])
print(test_dow)

## Step 6: 外部要因の比較

In [ ]:
print("="*80)
print("外部要因の比較")
print("="*80)

print("\n【CM実施状況】")
print(f"  Train     : {train_df['cm_flg'].sum()}回 (平均: {train_df['cm_flg'].mean():.3f}/日)")
print(f"  Validation: {val_df['cm_flg'].sum()}回 (平均: {val_df['cm_flg'].mean():.3f}/日)")
print(f"  Test      : {test_df['cm_flg'].sum()}回 (平均: {test_df['cm_flg'].mean():.3f}/日)")

print("\n【アカウント取得数】")
print(f"  Train     : 平均 {train_df['acc_get_cnt'].mean():.2f} (std: {train_df['acc_get_cnt'].std():.2f})")
print(f"  Validation: 平均 {val_df['acc_get_cnt'].mean():.2f} (std: {val_df['acc_get_cnt'].std():.2f})")
print(f"  Test      : 平均 {test_df['acc_get_cnt'].mean():.2f} (std: {test_df['acc_get_cnt'].std():.2f})")

print("\n【Google Trends検索数】")
print(f"  Train     : 平均 {train_df['search_cnt'].mean():.2f} (std: {train_df['search_cnt'].std():.2f})")
print(f"  Validation: 平均 {val_df['search_cnt'].mean():.2f} (std: {val_df['search_cnt'].std():.2f})")
print(f"  Test      : 平均 {test_df['search_cnt'].mean():.2f} (std: {test_df['search_cnt'].std():.2f})")

## Step 7: データの詳細分析

In [ ]:
# Validation期間とTest期間の全データを表示
print("="*80)
print("Validation期間の全データ")
print("="*80)
print(val_df[['cdr_date', 'dow', 'call_num', 'target_next_day', 'cm_flg', 'acc_get_cnt']].to_string())

print("\n" + "="*80)
print("Test期間の全データ")
print("="*80)
print(test_df[['cdr_date', 'dow', 'call_num', 'target_next_day', 'cm_flg', 'acc_get_cnt']].to_string())

## 結論と考察

In [ ]:
print("="*80)
print("調査結果まとめ")
print("="*80)

print("\n【観測されたスコア差】")
print("- Validation MAE: 42.61 (WAPE: 31.66%)")
print("- Test MAE: 27.05 (WAPE: 28.20%)")
print(f"- 差: {42.61 - 27.05:.2f} (Test の方が {(42.61 - 27.05) / 27.05 * 100:.1f}% 良い)")

print("\n【データの特徴】")
print(f"- Train平均: {train_mean:.2f}")
print(f"- Validation平均: {val_mean:.2f} (Train比: {(val_mean - train_mean) / train_mean * 100:+.1f}%)")
print(f"- Test平均: {test_mean:.2f} (Train比: {(test_mean - train_mean) / train_mean * 100:+.1f}%)")

print("\n【考えられる原因】")
print("")
print("1. データの時系列トレンド:")
print("   - Validation期間とTest期間で入電数の平均値や分散が異なる")
print("   - Validation期間が特殊な時期（年末年始、決算期など）に該当している可能性")
print("")
print("2. サンプル数の違い:")
print(f"   - Validation: {len(val_df)}件")
print(f"   - Test: {len(test_df)}件")
print("   - サンプル数が少ないと評価が不安定になる")
print("")
print("3. モデルの過学習:")
print("   - Validationセットで重みを最適化しているため、Validationに過学習")
print("   - しかし通常は過学習するとValidationの方が良くなるはず")
print("   - 今回はTestの方が良いので、データ分布の違いが主因と考えられる")
print("")
print("4. 季節性・ビジネスサイクル:")
print("   - 月別・曜日別の分析結果を確認すること")
print("   - 特定の月や曜日で入電パターンが大きく異なる可能性")

print("\n【推奨アクション】")
print("- 上記の可視化を確認し、Validation期間の特異性を把握")
print("- データ分割の期間を変更して検証（例: 1ヶ月ずつ、3ヶ月ずつなど）")
print("- Cross-validationを使って複数期間で評価")
print("- Testスコアを最終評価として信頼するべき")